## 1. Setup and Configuration

In [3]:
# === Imports & Configuration ===
import os
import warnings
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LogisticRegression

warnings.filterwarnings("ignore")

# Try to import LightGBM and XGBoost if available
try:
    import lightgbm as lgb
    LGB_AVAILABLE = True
except ImportError:
    lgb = None
    LGB_AVAILABLE = False
try:
    import xgboost as xgb
    XGB_AVAILABLE = True
except ImportError:
    xgb = None
    XGB_AVAILABLE = False

# === Directories ===
# Use local project structure. Set BASE_DIR to the current project root and prefer local `data/` and `train/` folders.
BASE_DIR = os.path.abspath(os.getcwd())
DATA_DIR = os.path.join(BASE_DIR, "data")
TRAIN_DIR = os.path.join(DATA_DIR, "train")
# Use the existing `results/` folder (already present in workspace) for outputs; fall back to a working folder if missing
OUT_DIR = os.path.join(BASE_DIR, "results")
CLEANED_DIR = os.path.join(OUT_DIR, "cleaned_originals")

os.makedirs(OUT_DIR, exist_ok=True)
os.makedirs(CLEANED_DIR, exist_ok=True)

# === Helper Functions ===
def save_csv(df, name):
    path = os.path.join(OUT_DIR, name)
    df.to_csv(path, index=False)
    print(f"✅ Saved {name} | shape: {df.shape} | path: {path}")
    return path

# A robust RMSE helper that works with different sklearn versions
from sklearn.metrics import mean_squared_error

def safe_rmse(y_true, y_pred):
    try:
        # newer sklearn supports squared=False
        return mean_squared_error(y_true, y_pred, squared=False)
    except TypeError:
        # fallback to sqrt of MSE
        return np.sqrt(mean_squared_error(y_true, y_pred))

def two_stage_score(y_true, y_pred):
    """Simple fallback score: RMSE on raw values. Keeps interface used later.
    Replace with task-specific metric if available.
    """
    y_true = np.asarray(y_true)
    y_pred = np.asarray(y_pred)
    # guard shapes
    if y_true.size == 0:
        return 0.0
    return float(np.sqrt(np.mean((y_true - y_pred) ** 2)))


## 2. Load All Datasets

In [4]:
# === Load all raw datasets (try train/, data/ and project root) ===
files = {
    # These files are expected in the 'train' subdirectory
    "new_house": "new_house_transactions.csv",
    "new_house_near": "new_house_transactions_nearby_sectors.csv",
    "pre_owned": "pre_owned_house_transactions.csv",
    "pre_owned_near": "pre_owned_house_transactions_nearby_sectors.csv",
    "land": "land_transactions.csv",
    "land_near": "land_transactions_nearby_sectors.csv",
    "poi": "sector_POI.csv",
    "city_search": "city_search_index.csv",
    "city_index": "city_indexes.csv",
    
    # These files may live in the data/ folder or project root
    "test": "test.csv",
    "sample_sub": "sample_submission.csv"
}

data = {}
for key, fname in files.items():
    # try candidate paths in order: train/, data/, project root
    candidates = [os.path.join(TRAIN_DIR, fname), os.path.join(DATA_DIR, fname), os.path.join(BASE_DIR, fname)]
    found = None
    for p in candidates:
        if os.path.exists(p):
            found = p
            break
    if found is not None:
        # use low_memory=False to avoid dtype inference warnings on large csvs
        df = pd.read_csv(found, low_memory=False)
        print(f"Loaded {fname} → {df.shape} from {found}")
        data[key] = df
    else:
        print(f"⚠️ Missing: {fname}. Tried: {candidates}")

# Quick dataset summary cell (separate cell for diagnostics)
print('\n=== Dataset summary ===')
for k, df in data.items():
    print(f"{k}: {df.shape} | columns: {list(df.columns)[:8]}{('...' if df.shape[1]>8 else '')}")


Loaded new_house_transactions.csv → (5433, 11) from C:\Users\Mitudru\Documents\ML Project\realestateprediction\data\train\new_house_transactions.csv
Loaded new_house_transactions_nearby_sectors.csv → (5360, 11) from C:\Users\Mitudru\Documents\ML Project\realestateprediction\data\train\new_house_transactions_nearby_sectors.csv
Loaded pre_owned_house_transactions.csv → (5360, 6) from C:\Users\Mitudru\Documents\ML Project\realestateprediction\data\train\pre_owned_house_transactions.csv
Loaded pre_owned_house_transactions_nearby_sectors.csv → (5427, 6) from C:\Users\Mitudru\Documents\ML Project\realestateprediction\data\train\pre_owned_house_transactions_nearby_sectors.csv
Loaded land_transactions.csv → (5896, 6) from C:\Users\Mitudru\Documents\ML Project\realestateprediction\data\train\land_transactions.csv
Loaded land_transactions_nearby_sectors.csv → (5025, 6) from C:\Users\Mitudru\Documents\ML Project\realestateprediction\data\train\land_transactions_nearby_sectors.csv
Loaded sector_PO

## 3. Data Cleaning

In [5]:
# === Clean Each Dataset Individually ===
cleaned = {}

# Generic cleaner
def clean_transactions(df):
    numcols = df.select_dtypes(include=[np.number]).columns
    df[numcols] = df[numcols].fillna(0)
    for c in df.select_dtypes(include=['object']).columns:
        df[c] = df[c].astype(str).str.strip()
    return df

for key in ["new_house","new_house_near","pre_owned","pre_owned_near","land","land_near"]:
    if key in data:
        df = clean_transactions(data[key])
        cleaned[key] = df
        save_csv(df, f"{key}_clean.csv")

# POI: Drop sparse cols, fill 0
poi = data["poi"]
miss = poi.isna().mean()
poi_clean = poi.drop(columns=miss[miss>0.7].index).fillna(0)
cleaned["poi"] = poi_clean
save_csv(poi_clean, "sector_POI_clean.csv")

# City Search Index
cs = data["city_search"]
cs_clean = cs.groupby("month", as_index=False)["search_volume"].sum().rename(columns={"search_volume":"city_search_volume"})
cleaned["city_search"] = cs_clean
save_csv(cs_clean, "city_search_index_clean.csv")

# City Index
ci = data["city_index"]
ci_clean = ci.dropna(thresh=int(ci.shape[0]*0.2), axis=1).fillna(method='ffill').fillna(method='bfill')
cleaned["city_index"] = ci_clean
save_csv(ci_clean, "city_indexes_clean.csv")


✅ Saved new_house_clean.csv | shape: (5433, 11) | path: C:\Users\Mitudru\Documents\ML Project\realestateprediction\results\new_house_clean.csv
✅ Saved new_house_near_clean.csv | shape: (5360, 11) | path: C:\Users\Mitudru\Documents\ML Project\realestateprediction\results\new_house_near_clean.csv
✅ Saved pre_owned_clean.csv | shape: (5360, 6) | path: C:\Users\Mitudru\Documents\ML Project\realestateprediction\results\pre_owned_clean.csv
✅ Saved pre_owned_near_clean.csv | shape: (5427, 6) | path: C:\Users\Mitudru\Documents\ML Project\realestateprediction\results\pre_owned_near_clean.csv
✅ Saved land_clean.csv | shape: (5896, 6) | path: C:\Users\Mitudru\Documents\ML Project\realestateprediction\results\land_clean.csv
✅ Saved land_near_clean.csv | shape: (5025, 6) | path: C:\Users\Mitudru\Documents\ML Project\realestateprediction\results\land_near_clean.csv
✅ Saved sector_POI_clean.csv | shape: (86, 142) | path: C:\Users\Mitudru\Documents\ML Project\realestateprediction\results\sector_POI_cl

'C:\\Users\\Mitudru\\Documents\\ML Project\\realestateprediction\\results\\city_indexes_clean.csv'

## 4. Merge and Aggregate

In [6]:
# === Merge All Cleaned Datasets ===

def aggregate(df):
    if "month" not in df.columns or "sector" not in df.columns:
        return None
    numcols = df.select_dtypes(include=[np.number]).columns
    return df.groupby(["month","sector"])[numcols].sum().reset_index()

agg_new = aggregate(cleaned["new_house"])
agg_pre = aggregate(cleaned["pre_owned"])
agg_land = aggregate(cleaned["land"])

merged = agg_new.copy()
def merge_pref(base, other, pref):
    if other is None: return base
    otherc = other.copy()
    for c in otherc.columns:
        if c not in ["month","sector"]:
            otherc.rename(columns={c: f"{pref}_{c}"}, inplace=True)
    return base.merge(otherc, on=["month","sector"], how="left")

merged = merge_pref(merged, agg_pre, "pre")
merged = merge_pref(merged, agg_land, "land")

# Add City & POI
merged = merged.merge(cleaned["city_search"], on="month", how="left")
merged = merged.merge(cleaned["poi"], on="sector", how="left")

save_csv(merged, "merged_full.csv")


✅ Saved merged_full.csv | shape: (5433, 161) | path: C:\Users\Mitudru\Documents\ML Project\realestateprediction\results\merged_full.csv


'C:\\Users\\Mitudru\\Documents\\ML Project\\realestateprediction\\results\\merged_full.csv'

## 5. Feature Engineering

In [7]:
# === Feature Engineering ===
m = merged.copy()
# Ensure month parsing is robust: try to convert month to datetime, otherwise keep as string
if "month" in m.columns:
    m["month_str"] = m["month"].astype(str)
    # pandas may not accept `infer_datetime_format` in some versions; keep only `errors`.
    m["month_dt"] = pd.to_datetime(m["month_str"], errors="coerce")
    # fall back to extracting number from formats like '2019-04' or '2019-Apr'
    m["month_num"] = m["month_dt"].dt.month.fillna(
        m["month_str"].str.extract(r"-(\d{1,2})$", expand=False).astype(float).fillna(0)
    ).astype(int)
    m["season"] = ((m["month_num"] % 12) // 3) + 1
else:
    m["month_str"] = ""
    m["month_num"] = 0
    m["season"] = 0

# Create lags and rolling means
for col in ["num_new_house_transactions","area_new_house_transactions","price_new_house_transactions","amount_new_house_transactions"]:
    if col in m.columns:
        for lag in [1,3,6]:
            m[f"{col}_lag{lag}"] = m.groupby("sector")[col].shift(lag).fillna(0)
        m[f"{col}_roll3"] = m.groupby("sector")[col].rolling(3, min_periods=1).mean().reset_index(level=0, drop=True)

# Supply-demand ratio
if "num_new_house_available_for_sale" in m.columns:
    m["supply_demand_ratio"] = m["num_new_house_available_for_sale"] / (m["num_new_house_transactions"] + 1)
    m["supply_demand_ratio"].replace(np.inf, 0, inplace=True)

# Target
if "amount_new_house_transactions" in m.columns:
    m["y_log1p"] = np.log1p(m["amount_new_house_transactions"].clip(lower=0))
else:
    m["y_log1p"] = 0

save_csv(m, "merged_engineered.csv")

# Diagnostics: compute correlations on numeric columns only (avoid ValueError from strings like '2019-Apr')
if "amount_new_house_transactions" in m.columns:
    numeric = m.select_dtypes(include=[np.number])
    if "amount_new_house_transactions" in numeric.columns:
        target = "amount_new_house_transactions"
        corr = numeric.corr()[target].abs().sort_values(ascending=False)
        print('\nTop correlated features:')
        print(corr.head(20))
    else:
        print('\nTarget is not numeric in numeric subset, skipping correlation')
else:
    print('\nTarget column not found in merged, skipping correlation')


✅ Saved merged_engineered.csv | shape: (5433, 183) | path: C:\Users\Mitudru\Documents\ML Project\realestateprediction\results\merged_engineered.csv

Top correlated features:
amount_new_house_transactions            1.000000
area_new_house_transactions              0.871142
amount_new_house_transactions_roll3      0.847126
num_new_house_transactions               0.846628
area_new_house_transactions_roll3        0.736229
num_new_house_transactions_roll3         0.716164
y_log1p                                  0.693480
amount_new_house_transactions_lag1       0.583104
area_new_house_transactions_lag1         0.570533
num_new_house_transactions_lag1          0.557265
amount_new_house_transactions_lag3       0.511882
area_new_house_transactions_lag3         0.507815
area_new_house_available_for_sale        0.499343
num_new_house_transactions_lag3          0.495415
amount_new_house_transactions_lag6       0.495047
num_new_house_available_for_sale         0.492418
area_new_house_transaction

## 6. Model Training (two stage)

In [9]:
# === Two-Stage Model: Classifier + Regressor ===

if "amount_new_house_transactions" in m.columns:
    # use np.where to avoid static-analysis warnings and ensure integer dtype
    m["is_zero"] = np.where(m["amount_new_house_transactions"] == 0, 1, 0).astype('int8')
else:
    # create zero column so downstream code doesn't break
    m["is_zero"] = np.zeros(len(m), dtype='int8')

# Split time-based
months = sorted(m["month"].dropna().unique()) if "month" in m.columns else []
val_months = months[-6:] if len(months) > 0 else []
train_mask = ~m["month"].isin(val_months) if len(val_months) > 0 else np.ones(len(m), dtype=bool)
val_mask = m["month"].isin(val_months) if len(val_months) > 0 else np.zeros(len(m), dtype=bool)

# Use only numeric features for training
X = m.select_dtypes(include=[np.number]).drop(columns=[c for c in ["y_log1p"] if c in m.columns], errors='ignore')
y_clf = m["is_zero"]
y_reg = m["y_log1p"]

X_train, X_val = X[train_mask], X[val_mask]
y_clf_train, y_clf_val = y_clf[train_mask], y_clf[val_mask]
y_reg_train, y_reg_val = y_reg[train_mask], y_reg[val_mask]

# --- Classifier ---
if LGB_AVAILABLE:
    dtrain = lgb.Dataset(X_train, label=y_clf_train)
    dval = lgb.Dataset(X_val, label=y_clf_val)
    
    params_clf = {
        "objective": "binary",
        "metric": "auc",
        "verbosity": -1,
        "learning_rate": 0.05,
        "num_leaves": 64,
        "feature_fraction": 0.8,
        "bagging_fraction": 0.8,
        "seed": 42
    }
    
    # Use callback-based early stopping (works for all versions)
    clf = lgb.train(
        params_clf,
        dtrain,
        num_boost_round=500,
        valid_sets=[dtrain, dval],
        valid_names=["train", "val"],
        callbacks=[lgb.early_stopping(stopping_rounds=50), lgb.log_evaluation(100)]
    )
    p_zero_val = clf.predict(X_val, num_iteration=clf.best_iteration)
else:
    clf = LogisticRegression(max_iter=1000)
    clf.fit(X_train, y_clf_train)
    p_zero_val = clf.predict_proba(X_val)[:, 1]

# --- Regressor ---
if LGB_AVAILABLE:
    dtrain_r = lgb.Dataset(X_train, label=y_reg_train)
    dval_r = lgb.Dataset(X_val, label=y_reg_val)
    
    params_reg = {
        "objective": "regression",
        "metric": "mae",
        "verbosity": -1,
        "learning_rate": 0.03,
        "num_leaves": 128,
        "feature_fraction": 0.8,
        "bagging_fraction": 0.8,
        "seed": 42
    }
    
    reg = lgb.train(
        params_reg,
        dtrain_r,
        num_boost_round=1000,
        valid_sets=[dtrain_r, dval_r],
        valid_names=["train", "val"],
        callbacks=[lgb.early_stopping(stopping_rounds=50), lgb.log_evaluation(100)]
    )
    
    p_reg_val = np.expm1(reg.predict(X_val, num_iteration=reg.best_iteration))
else:
    reg = RandomForestRegressor(n_estimators=300, random_state=42, n_jobs=-1)
    reg.fit(X_train, y_reg_train)
    p_reg_val = np.expm1(reg.predict(X_val))

# --- Final predictions ---
p_final_val = (1 - p_zero_val) * p_reg_val
y_true = m.loc[val_mask, "amount_new_house_transactions"].values

score = two_stage_score(y_true, p_final_val)
mae = mean_absolute_error(y_true, p_final_val)
rmse = safe_rmse(y_true, p_final_val)

print(f"✅ Validation Score: {score:.5f} | MAE: {mae:.2f} | RMSE: {rmse:.2f}")


Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[1]	train's auc: 1	val's auc: 1
Training until validation scores don't improve for 50 rounds
[100]	train's l1: 0.0612232	val's l1: 0.059071
[200]	train's l1: 0.00837149	val's l1: 0.014474
[300]	train's l1: 0.00547851	val's l1: 0.0139353
[400]	train's l1: 0.00403634	val's l1: 0.0139093
Early stopping, best iteration is:
[381]	train's l1: 0.00425181	val's l1: 0.0138899
✅ Validation Score: 2216.40549 | MAE: 432.68 | RMSE: 2216.41


## 7. Final Submission File

In [10]:
# === Predict on Test and Build Submission (FINAL FIXED VERSION) ===

# Keep original test copy before feature alignment
test_original = data["test"].copy()

# --- Ensure feature alignment ---
train_features = X.columns.tolist()  # features used during training
missing_in_test = [f for f in train_features if f not in test_original.columns]
extra_in_test = [f for f in test_original.columns if f not in train_features]

# Fill missing training features in test with 0
for f in missing_in_test:
    test_original[f] = 0

# Create aligned version for prediction
X_test = test_original[train_features].fillna(0)

print(f"✅ Test alignment done: {len(train_features)} features used.")
print(f"   → Added {len(missing_in_test)} missing columns.")
print(f"   → Dropped {len(extra_in_test)} extra columns.")

# --- Predictions ---
if LGB_AVAILABLE:
    p_zero_test = clf.predict(X_test, num_iteration=getattr(clf, "best_iteration", None))
    p_reg_test = np.expm1(reg.predict(X_test, num_iteration=getattr(reg, "best_iteration", None)))
else:
    p_zero_test = clf.predict_proba(X_test)[:, 1]
    p_reg_test = np.expm1(reg.predict(X_test))

# --- Combine Two-Stage Predictions ---
p_final_test = (1 - p_zero_test) * p_reg_test
p_final_test = np.clip(p_final_test, 0, None)
p_final_test = np.where(p_final_test < 1.0, 0.0, p_final_test)

# --- Build Submission ---
if "id" in data["test"].columns:
    # ✅ Normal case: Kaggle test file includes `id`
    sub = pd.DataFrame({
        "id": data["test"]["id"],
        "new_house_transaction_amount": p_final_test
    })
else:
    # ✅ Fallback: reconstruct from month & sector (if available)
    if {"month", "sector"}.issubset(data["test"].columns):
        sub = pd.DataFrame({
            "id": data["test"]["month"].astype(str) + "_sector " + data["test"]["sector"].astype(str),
            "new_house_transaction_amount": p_final_test
        })
    else:
        # ✅ Ultimate fallback if no id/month/sector available
        sub = pd.DataFrame({
            "id": [f"sample_{i}" for i in range(len(p_final_test))],
            "new_house_transaction_amount": p_final_test
        })

save_csv(sub, "submission.csv")
print("🏁 Submission file ready! Upload to Kaggle.")


✅ Test alignment done: 179 features used.
   → Added 179 missing columns.
   → Dropped 2 extra columns.
✅ Saved submission.csv | shape: (1152, 2) | path: C:\Users\Mitudru\Documents\ML Project\realestateprediction\results\submission.csv
🏁 Submission file ready! Upload to Kaggle.
